In [18]:
# 導入必要的套件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

# 建立 Ollama 模型
model = ChatOllama(
    model="gpt-oss:20b",
    temperature=0.7
)


In [19]:
# 定義主要的提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human", "請分析這家餐廳：{restaurant_info}"),
    ]
)


In [20]:
pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的優點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的優點。"),
        ]
)

pros_branch_chain = (
    pros_template
    | model
    | StrOutputParser()
)

In [21]:
cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的缺點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的缺點。"),
        ]
)

cons_branch_chain =(
    cons_template
    | model
    | StrOutputParser() 
    
)

In [22]:
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"""
🍽️ 餐廳評論分析報告
{'='*50}

✅ 優點分析：
{pros}

❌ 缺點分析：
{cons}

📊 總結：這是一個平衡的餐廳評論分析
"""

In [23]:
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableLambda(lambda x:{"restaurant_info":x})
    | RunnableParallel(
        pros = pros_branch_chain,
        cons = cons_branch_chain
    )
    | RunnableLambda(lambda x: combine_pros_cons(x['pros'],x['cons']))
)

In [24]:
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_info": restaurant_info})

print(result)



🍽️ 餐廳評論分析報告

✅ 優點分析：
**這家台北市中心義大利餐廳的優點總結：**

- **優雅舒適的環境**  
  乾淨、寬敞的空間，木質與歐式古典裝潢，柔和燈光與適度音樂營造出浪漫、安靜的用餐氛圍，讓客人能專注於美食與互動。

- **親切專業的服務**  
  服務人員熟悉菜單，能即時提供中餐與酒水搭配建議，讓非義式料理愛好者也能安心點餐並享受個性化建議。

- **高品質正宗義大利料理**  
  主打炭烤牛排、海鮮燉飯、手工意粉等料理，食材新鮮、口味純正，堅守義大利傳統風味。

- **適合多種場合**  
  典雅氛圍與相對安靜的用餐區，特別適合情侶約會、商務聚餐、生日慶祝等重要場合。

- **豐富多樣的酒水選擇**  
  提供多款義大利紅白葡萄酒與手工啤酒，能與餐點完美搭配，提升整體味覺層次。

- **服務細節提升用餐體驗**  
  服務人員會在餐前提供酒水搭配建議與餐後甜點推薦，並可針對常客或會員提供專屬優惠，提升客戶忠誠度。

這些優點共同營造出一個既能享受正宗義大利風味，又能感受台北市中心獨特氛圍的高品質用餐場所。

❌ 缺點分析：
**餐廳缺點（總結）**

1. **價格偏高**  
   - 以台北市中心標準來看，單點與套餐價格均屬高價位，可能讓預算有限的客群望而卻步。

2. **等待時間長**  
   - 週末與晚餐高峰期常需排隊 30–60 分鐘，影響用餐時間規劃，對急於就餐者不利。

3. **份量相對較小**  
   - 主菜份量小，配菜亦偏少，容易讓饑餓的客人感到不滿，尤其是喜歡大份量的消費者。

4. **菜單選擇有限**  
   - 雖以傳統義大利料理為主，但創新菜品與素食選項有限，可能無法滿足多元飲食需求。

---

以上四項為此餐廳最主要的缺點，建議在預約或選擇時加以考量。

📊 總結：這是一個平衡的餐廳評論分析

